# Primer parcial ejercicio D1(Clasificacion one-vs-all) Grupo 1
 <h3>En este laboratorio se hizo el uso del dataset para aplicar clasificacion One-vs-All<h3>
 <HR>
 <h3>
  NOMBRE: POLO ORELLANA BRAYAN SIMON <br>
  CARRERA: INGENIERIA DE SISTEMAS <BR>
  FECHA: 09/04/2024 <BR>

  * [Enlace de invitacion para ser colaborador](https://github.com/bspoloo/SIS420-012024/invitations)
  
  * [Enlace al git hub](https://github.com/bspoloo/SIS420-012024/tree/main/Parciales/Primer%20Parcial)
  
  * [Enlace al Colab](https://colab.research.google.com/drive/1A5Sokh74ki7iit0AfevzdfGFPh4aIJ03?authuser=2)
 <h3>


Para el laboratorio se hizo uso del para entrenar el modelo de **Regresion Logistica one vs all** aplicando regularizacion y dada la información a los sintomas de una persona que pudo haaber sufrido covid-19, se creo un modelo de aprendizaje automático que pueda predecir si el paciente tiene un alto riesgo o no.

Se quiere construir un modelo que, dados los síntomas, el estado y el historial medico actuales de un paciente con Covid-19, prediga si el paciente tiene un alto riesgo o no, haciendo uso del dataset: [COVID-19 Dataset](https://www.kaggle.com/datasets/meirnizri/covid19-dataset).

El archivo `Covid Data.csv` contiene un conjunto de datos necesario para el entrenamniento del modelo.

# Explicacion del dataset:

Para el dataset D1 se analizo lo que el dataset trata de buscar, en este caso el objetivo principal de este proyecto es construir un modelo de aprendizaje automático que, dados los síntomas, el estado y el historial médico actuales de un paciente con Covid-19, prediga si el paciente tiene un alto riesgo o no, de acuerdo a sus niveles de riesgo de 1 al 7. dicho se llego a la conclucion de que lo mas apropiado seria **clasificacion one-vs-all** por los niveles a los que puede pertenecer un paciente.

**Variables de entrada(X):**


- sex (Sexo): 1 para mujer y 2 para hombre
-age (Edad): del paciente
-patient type (Tipo de paciente): tipo de atención que recibió el paciente en la -unidad. 1 para regreso a casa y 2 para hospitalización.
-pneumonia (Neumonía): si el paciente ya tiene inflamación de los sacos aéreos o -no.
-pregnancy (Embarazo): si el paciente está embarazada o no.
-diabetes (Diabetes): si el paciente tiene diabetes o no.
-copd (EPOC): Indica si el paciente tiene enfermedad pulmonar obstructiva -crónica o no.
-asthma (Asma): si el paciente tiene asma o no.
-inmsupr (Inmunodeprimido): si el paciente está inmunosuprimido o no.
-hypertension (Hipertensión): si el paciente tiene hipertensión o no.
-cardiovascular (Cardiovascular): si el paciente tiene una enfermedad -relacionada con el corazón o los vasos sanguíneos.
-renal chronic (Insuficiencia renal crónica): si el paciente tiene una -enfermedad renal crónica o no.
-other disease (Otras enfermedades): si el paciente tiene otras enfermedades o -no.
-obesity (Obesidad): si el paciente tiene obesidad o no.
-tobacco (Tabaco): si el paciente es usuario de tabaco.
-usmr (USMR): Indica si el paciente fue atendido en unidades médicas de primer, -segundo o tercer nivel.
-medical unit (Unidad médica): tipo de institución del Sistema Nacional de Salud -que brindó la atención.
-intubed (Intubado): si el paciente fue conectado al ventilador.
-icu (UCI): Indica si el paciente fue admitido en una Unidad de Cuidados -Intensivos.
-date died (Fecha de fallecimiento): Si el paciente falleció, indique la fecha de fallecimiento, y 9999-99-99 en caso contrario.


**Variables de salida(Y):**


- classification (Clasificación): resultado de la prueba de covid. Los valores del 1 al 3 significan que el paciente fue diagnosticado con covid en diferentes grados. 4 o más significa que el paciente no es portador de covid o que la prueba es inconcluyente.


## 1. Clasificacion one-vs-all

Primero se importo todas las librerias necesarias:

In [ ]:
# utilizando la libreria os para manejos de directorios
import os

# Computacion vectorial y cientifica para python
import numpy as np

#importamos pandas para el manejo del dataset, y separarlos dentro de una matriz
import pandas as pd

#esta tabulate nos sirve para hacer tablas
from tabulate import tabulate

# Librerias para graficación (trazado de gráficos)
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # -> Necesario para graficar superficies 3D

#Para separa el 20% y 80%
from sklearn.model_selection import train_test_split

#para poder usar datos de tipo datetime en pandas
from datetime import datetime

# Modulo de optimizacion en scipy
from scipy import optimize

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

### 1.1 Cargamos los datos del dataset
cargamos los datos haciendo el uso de la libreria **Pandas** que  es una herramienta poderosa y versátil utilizada para manipulación y análisis de datos. Ofrece estructuras de datos flexibles y eficientes para trabajar con datos tabulares, como hojas de cálculo en Excel o tablas SQL.


In [ ]:
#Haciendo uso de la libreria pandas para leer el dataset, delimitado por ","

# dataset = pd.read_csv('Covid Data.csv', delimiter=',')
#para usarlo a travez de google colab
dataset = pd.read_csv('/content/gdrive/MyDrive/Datasets/Covid Data.csv', delimiter=',')


#ahora tenemos un dataframe
#Imprimimos en una tabla el dataset para hacer un analisis mas claro.
dataset

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,13,2,1,9999-99-99,97,2,40,97,2,...,2,2,2,2,2,2,2,2,7,97
1048571,1,13,2,2,9999-99-99,2,2,51,97,2,...,2,2,1,2,2,2,2,2,7,2
1048572,2,13,2,1,9999-99-99,97,2,55,97,2,...,2,2,2,2,2,2,2,2,7,97
1048573,2,13,2,1,9999-99-99,97,2,28,97,2,...,2,2,2,2,2,2,2,2,7,97


Tratamos los datos para la fecha de muerte o `DATE_DIED` ya que solo se pone la fecha para este en caso de que el paciente hay muerto o no, entonces cambiamos a 1 para decir si esta muerto o 0 para decir que aun sigue vivo:

### 1.2 Analisis del dataset

Hacemos un analisis del dataset mostrando su informacion usando la funcion de `info()`:

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   USMER                 1048575 non-null  int64 
 1   MEDICAL_UNIT          1048575 non-null  int64 
 2   SEX                   1048575 non-null  int64 
 3   PATIENT_TYPE          1048575 non-null  int64 
 4   DATE_DIED             1048575 non-null  object
 5   INTUBED               1048575 non-null  int64 
 6   PNEUMONIA             1048575 non-null  int64 
 7   AGE                   1048575 non-null  int64 
 8   PREGNANT              1048575 non-null  int64 
 9   DIABETES              1048575 non-null  int64 
 10  COPD                  1048575 non-null  int64 
 11  ASTHMA                1048575 non-null  int64 
 12  INMSUPR               1048575 non-null  int64 
 13  HIPERTENSION          1048575 non-null  int64 
 14  OTHER_DISEASE         1048575 non-null  int64 
 15

Se puede observar que el `DATE_DIED` es de tipo de dato object, lo cual no necesitamos, debemos convertirlo a un dato entero.

Para eso tomaremos la fecha actual y restarla con el `DATE_DIED` y convertirlo a dias de tipo entero:

#### 1.2.1 Tratado de datos

Primero convertimos todos los `9999-99-99` a la fecha actual, esto se supone porque el paciente aun sigue vivo:

In [ ]:
dataset['DATE_DIED'] = dataset['DATE_DIED'].replace('9999-99-99', datetime.today().strftime('%d/%m/%Y'))

# Convertir la columna 'DATE_DIED' al formato de fecha de Pandas
dataset['DATE_DIED'] = pd.to_datetime(dataset['DATE_DIED'], format='%d/%m/%Y', errors='coerce')
# Eliminar las filas donde no se pudo convertir la fecha
dataset = dataset.dropna(subset=['DATE_DIED'])

#imprimimos el dataset para ver los cambios
dataset

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,2020-05-03,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,2020-06-03,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,2020-06-09,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,2020-06-12,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,2020-06-21,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,13,2,1,2024-04-09,97,2,40,97,2,...,2,2,2,2,2,2,2,2,7,97
1048571,1,13,2,2,2024-04-09,2,2,51,97,2,...,2,2,1,2,2,2,2,2,7,2
1048572,2,13,2,1,2024-04-09,97,2,55,97,2,...,2,2,2,2,2,2,2,2,7,97
1048573,2,13,2,1,2024-04-09,97,2,28,97,2,...,2,2,2,2,2,2,2,2,7,97


Ahora tenemos que calcular los dias que pasaron con respecto a la fecha actual:

In [ ]:
dataset['DATE_DIED'] = (datetime.today() - dataset['DATE_DIED']).dt.days
dataset['DATE_DIED'] = dataset['DATE_DIED'] / 365

dataset

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,3.936986,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,3.852055,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,3.835616,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,3.827397,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,3.802740,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,13,2,1,0.000000,97,2,40,97,2,...,2,2,2,2,2,2,2,2,7,97
1048571,1,13,2,2,0.000000,2,2,51,97,2,...,2,2,1,2,2,2,2,2,7,2
1048572,2,13,2,1,0.000000,97,2,55,97,2,...,2,2,2,2,2,2,2,2,7,97
1048573,2,13,2,1,0.000000,97,2,28,97,2,...,2,2,2,2,2,2,2,2,7,97


mostrando la tabla nos damos cuenta que nuestra Y a predicir es el **CLASFFICATION_FINAL**.

Para tener una mejor vision de la cantidad de clases que existe, se hizo el siguiente codigo:

donde separamos nuestra columna Y con `value_counts()` este método de pandas cuenta el número de veces que aparece cada valor único en la columna **Credit_Score** del DataFrame. Devuelve una Serie pandas donde los índices son los valores únicos de la columna **Credit_Score** y los valores son el recuento de ocurrencias de cada valor.

In [ ]:
#para contar cuantas clases contiene nuestra columna y
class_counts = dataset["CLASIFFICATION_FINAL"].value_counts()

#para contar cuantas caracteristicas tiene nuestro dataset, obviamente con sin contar nuestra y, por eso lo dropeamos, tambien dropeamos la primera columna que no tiene nombre
feactures_counts =dataset.drop(['CLASIFFICATION_FINAL'], axis=1)
feactures_counts = feactures_counts.shape[1]


print(f"la cantidad de caracteristicas es: { feactures_counts}")
print(f'la cantidad de clases es: 7')

#mostramos la cantidad de clases tiene, y en que cantidad
class_counts

la cantidad de caracteristicas es: 20
la cantidad de clases es: 7


CLASIFFICATION_FINAL
7    499250
3    381527
6    128133
5     26091
1      8601
4      3122
2      1851
Name: count, dtype: int64

Se nos muestra en orden, donde el tipo de grado 7 hay mas pacientes, mientras que del grado 2 hay una minima cantidad.

Una vez tratado todos los datos, Hacemos un nuevo analisis del dataset mostrando su informacion usando la funcion de `info()`, donde se nos muestra que todos los datos ahora son de tipo entero:

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   USMER                 1048575 non-null  int64  
 1   MEDICAL_UNIT          1048575 non-null  int64  
 2   SEX                   1048575 non-null  int64  
 3   PATIENT_TYPE          1048575 non-null  int64  
 4   DATE_DIED             1048575 non-null  float64
 5   INTUBED               1048575 non-null  int64  
 6   PNEUMONIA             1048575 non-null  int64  
 7   AGE                   1048575 non-null  int64  
 8   PREGNANT              1048575 non-null  int64  
 9   DIABETES              1048575 non-null  int64  
 10  COPD                  1048575 non-null  int64  
 11  ASTHMA                1048575 non-null  int64  
 12  INMSUPR               1048575 non-null  int64  
 13  HIPERTENSION          1048575 non-null  int64  
 14  OTHER_DISEASE         1048575 non-

Ahora tenemos que convertir todos los datos que sean de la clase 7, ahora que pasen a pertencer a la clase 0:

In [ ]:
dataset['CLASIFFICATION_FINAL'] = dataset['CLASIFFICATION_FINAL'].replace(7, 0)
dataset

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,3.936986,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,3.852055,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,3.835616,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,3.827397,97,2,53,2,2,...,2,2,2,2,2,2,2,2,0,97
4,2,1,2,1,3.802740,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2,13,2,1,0.000000,97,2,40,97,2,...,2,2,2,2,2,2,2,2,0,97
1048571,1,13,2,2,0.000000,2,2,51,97,2,...,2,2,1,2,2,2,2,2,0,2
1048572,2,13,2,1,0.000000,97,2,55,97,2,...,2,2,2,2,2,2,2,2,0,97
1048573,2,13,2,1,0.000000,97,2,28,97,2,...,2,2,2,2,2,2,2,2,0,97


Volviendo a mostrar la cantidad de clases:

In [ ]:
#para contar cuantas clases contiene nuestra columna y
class_counts = dataset["CLASIFFICATION_FINAL"].value_counts()

#para contar cuantas caracteristicas tiene nuestro dataset, obviamente con sin contar nuestra y, por eso lo dropeamos, tambien dropeamos la primera columna que no tiene nombre
feactures_counts =dataset.drop(['CLASIFFICATION_FINAL'], axis=1)
feactures_counts = feactures_counts.shape[1]

#almacenamos la cantidad de caracteristicas en una variable
input_layer_size  = feactures_counts;

#Podemos tomar como 8 tipos de clases, o transformar todas los valores 7 a 0
num_labels = 7;


print(f"la cantidad de caracteristicas es: { feactures_counts}")
print(f'la cantidad de clases es: 7')

class_counts

la cantidad de caracteristicas es: 20
la cantidad de clases es: 7


CLASIFFICATION_FINAL
0    499250
3    381527
6    128133
5     26091
1      8601
4      3122
2      1851
Name: count, dtype: int64

### 1.3 Separacion de datos
Se debe tomar en cuenta que cada clase tiene su propia cantidad, por lo cual separarlos directamente en un 80% para entrenamiento y un 20% para test no seria tan efectivo, ya que puede que en el 80% hay mas datos de una clase que las otras, provocando que nuestro modelo no conozca mucho sobre esa clase, por lo cual debe separarse un 80% para entrenamiento y un 20% para pruebas de cada clase.

Para este hacemos uso de la libreria **train_test_split**

In [ ]:
# hacemos uso del DataFrame llamado 'dataset' que contiene nuestros datos datos
# y es la columna que contiene las etiquetas de clase (en este caso, 'CLASIFFICATION_FINAL')

#creamos una variable temporal que contentra toda la columna de 'CLASIFFICATION_FINAL'
y_temp = dataset['CLASIFFICATION_FINAL']

# Para la clase 0
#donde y_temp es igual a 1, separamos los datos en train_class_1, test_class_1
data_class_0 = dataset[y_temp == 0]
train_class_0, test_class_0 = train_test_split(data_class_0, test_size=0.2, random_state=42)

# Para la clase 1
#donde y_temp es igual a 1, separamos los datos en train_class_1, test_class_1
data_class_1 = dataset[y_temp == 1]
train_class_1, test_class_1 = train_test_split(data_class_1, test_size=0.2, random_state=42)

# Para la clase 2
#donde y_temp es igual a 2, separamos los datos en train_class_2, test_class_2
data_class_2 = dataset[y_temp == 2]
train_class_2, test_class_2 = train_test_split(data_class_2, test_size=0.2, random_state=42)

# Para la clase 3
#donde y_temp es igual a 3, separamos los datos en train_class_3, test_class_3
data_class_3 = dataset[y_temp == 3]
train_class_3, test_class_3 = train_test_split(data_class_3, test_size=0.2, random_state=42)

# Para la clase 4
#donde y_temp es igual a 4, separamos los datos en train_class_4, test_class_4
data_class_4 = dataset[y_temp == 4]
train_class_4, test_class_4 = train_test_split(data_class_4, test_size=0.2, random_state=42)

# Para la clase 5
#donde y_temp es igual a 5, separamos los datos en train_class_5, test_class_5
data_class_5 = dataset[y_temp == 5]
train_class_5, test_class_5 = train_test_split(data_class_5, test_size=0.2, random_state=42)

# Para la clase 6
#donde y_temp es igual a 6, separamos los datos en train_class_6, test_class_6
data_class_6 = dataset[y_temp == 6]
train_class_6, test_class_6 = train_test_split(data_class_6, test_size=0.2, random_state=42)


haciendo conteo de separacion de datos:

In [ ]:
print(f"Para la clase 0 tenemos una cantidad de: { data_class_0.shape[0]} donde el 80% es: {train_class_0.shape[0]} y el 20% es: {test_class_0.shape[0]}")
print(f"Para la clase 1 tenemos una cantidad de: { data_class_1.shape[0]} donde el 80% es: {train_class_1.shape[0]} y el 20% es: {test_class_1.shape[0]}")
print(f"Para la clase 2 tenemos una cantidad de: { data_class_2.shape[0]} donde el 80% es: {train_class_2.shape[0]} y el 20% es: {test_class_2.shape[0]}")
print(f"Para la clase 3 tenemos una cantidad de: { data_class_3.shape[0]} donde el 80% es: {train_class_3.shape[0]} y el 20% es: {test_class_3.shape[0]}")
print(f"Para la clase 4 tenemos una cantidad de: { data_class_4.shape[0]} donde el 80% es: {train_class_4.shape[0]} y el 20% es: {test_class_4.shape[0]}")
print(f"Para la clase 5 tenemos una cantidad de: { data_class_5.shape[0]} donde el 80% es: {train_class_5.shape[0]} y el 20% es: {test_class_5.shape[0]}")
print(f"Para la clase 6 tenemos una cantidad de: { data_class_6.shape[0]} donde el 80% es: {train_class_6.shape[0]} y el 20% es: {test_class_6.shape[0]}")
print(f"La cantidad total de datos es: {dataset.shape[0]}")

Para la clase 0 tenemos una cantidad de: 499250 donde el 80% es: 399400 y el 20% es: 99850
Para la clase 1 tenemos una cantidad de: 8601 donde el 80% es: 6880 y el 20% es: 1721
Para la clase 2 tenemos una cantidad de: 1851 donde el 80% es: 1480 y el 20% es: 371
Para la clase 3 tenemos una cantidad de: 381527 donde el 80% es: 305221 y el 20% es: 76306
Para la clase 4 tenemos una cantidad de: 3122 donde el 80% es: 2497 y el 20% es: 625
Para la clase 5 tenemos una cantidad de: 26091 donde el 80% es: 20872 y el 20% es: 5219
Para la clase 6 tenemos una cantidad de: 128133 donde el 80% es: 102506 y el 20% es: 25627
La cantidad total de datos es: 1048575


Separamos cada uno en sus **X_train**, **y_train**, **X_test** y **y_test** respectivos:

In [ ]:
#para la parte de entrenamiento, separamos las caracteristicas de la etiqueta

#para la clase 0
X_train_class_0 = train_class_0.drop(['CLASIFFICATION_FINAL'], axis=1)
y_train_class_0 = train_class_0['CLASIFFICATION_FINAL']

#para la clase 1
X_train_class_1 = train_class_1.drop(['CLASIFFICATION_FINAL'], axis=1)
y_train_class_1 = train_class_1['CLASIFFICATION_FINAL']

#para la clase 2
X_train_class_2 = train_class_2.drop(['CLASIFFICATION_FINAL'], axis=1)
y_train_class_2 = train_class_2['CLASIFFICATION_FINAL']

#para la clase 3
X_train_class_3 = train_class_3.drop(['CLASIFFICATION_FINAL'], axis=1)
y_train_class_3 = train_class_3['CLASIFFICATION_FINAL']

#para la clase 4
X_train_class_4 = train_class_4.drop(['CLASIFFICATION_FINAL'], axis=1)
y_train_class_4 = train_class_4['CLASIFFICATION_FINAL']

#para la clase 5
X_train_class_5 = train_class_5.drop(['CLASIFFICATION_FINAL'], axis=1)
y_train_class_5 = train_class_5['CLASIFFICATION_FINAL']

#para la clase 6
X_train_class_6 = train_class_6.drop(['CLASIFFICATION_FINAL'], axis=1)
y_train_class_6 = train_class_6['CLASIFFICATION_FINAL']


#ahora para la parte de pruebas, separamos las caracteristicas de la etiqueta

#para la clase 0
X_test_class_0 = test_class_0.drop(['CLASIFFICATION_FINAL'], axis=1)
y_test_class_0 = test_class_0['CLASIFFICATION_FINAL']

#para la clase 1
X_test_class_1 = test_class_1.drop(['CLASIFFICATION_FINAL'], axis=1)
y_test_class_1 = test_class_1['CLASIFFICATION_FINAL']

#para la clase 2
X_test_class_2 = test_class_2.drop(['CLASIFFICATION_FINAL'], axis=1)
y_test_class_2 = test_class_2['CLASIFFICATION_FINAL']

#para la clase 3
X_test_class_3 = test_class_3.drop(['CLASIFFICATION_FINAL'], axis=1)
y_test_class_3 = test_class_3['CLASIFFICATION_FINAL']

#para la clase 4
X_test_class_4 = test_class_4.drop(['CLASIFFICATION_FINAL'], axis=1)
y_test_class_4 = test_class_4['CLASIFFICATION_FINAL']

#para la clase 5
X_test_class_5 = test_class_5.drop(['CLASIFFICATION_FINAL'], axis=1)
y_test_class_5 = test_class_5['CLASIFFICATION_FINAL']

#para la clase 6
X_test_class_6 = test_class_6.drop(['CLASIFFICATION_FINAL'], axis=1)
y_test_class_6 = test_class_6['CLASIFFICATION_FINAL']

ahora unimos todos en una sola matriz para X y y:
pero luego debemos mezclar los datos, haciendo uso de ``np.random.permutation(len(X))`` genera un arreglo de índices permutados aleatoriamente.
Luego, estos índices se usan para reorganizar tanto las características como las etiquetas de entrenamiento y prueba.

In [ ]:
#separando los datos de entrenamiento y pruebas

#para los datos de entrenamiento
X_train = pd.concat([X_train_class_0,X_train_class_1, X_train_class_2, X_train_class_3, X_train_class_4, X_train_class_5, X_train_class_6]).values
y_train = pd.concat([y_train_class_0,y_train_class_1, y_train_class_2, y_train_class_3, y_train_class_4, y_train_class_5, y_train_class_6]).values

indices_train = np.random.permutation(len(X_train))
X_train = X_train[indices_train]
y_train = y_train[indices_train]
m_train = len(y_train)

#para los datos de pruebas
X_test = pd.concat([X_test_class_0,X_test_class_1, X_test_class_2, X_test_class_3, X_test_class_4, X_test_class_5, X_test_class_6]).values
y_test = pd.concat([y_test_class_0,y_test_class_1, y_test_class_2, y_test_class_3, y_test_class_4, y_test_class_5, y_test_class_6]).values

indices_test = np.random.permutation(len(X_test))
X_test = X_test[indices_test]
y_test = y_test[indices_test]
m_test = len(y_test)

Visualizamos algunos datos:

In [ ]:
print('{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}'.format(
    'X[:,0]', 'X[:, 1]', 'X[:, 2]', 'X[:, 3]', 'X[:, 4]', 'X[:, 5]', 'X[:, 6]', 'X[:, 7]', 'X[:, 8]', 'X[:, 9]', 'X[:, 10]', 'X[:, 11]', 'X[:, 12]',
    'X[:, 13]', 'X[:, 14]', 'X[:, 15]', 'X[:, 16]', 'X[:, 17]', 'X[:, 18]', 'X[:, 19]','Y'
))
print('-' * 250)

for i in range(10):
    print('{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}{:10.0f}'.format(
        X_train[i, 0],
        X_train[i, 1],
        X_train[i, 2],
        X_train[i, 3],
        X_train[i, 4],
        X_train[i, 5],
        X_train[i, 6],
        X_train[i, 7],
        X_train[i, 8],
        X_train[i, 9],
        X_train[i, 10],
        X_train[i, 11],
        X_train[i, 12],
        X_train[i, 13],
        X_train[i, 14],
        X_train[i, 15],
        X_train[i, 16],
        X_train[i, 17],
        X_train[i, 18],
        X_train[i, 19],
        y_train[i]
    ))

print(" ")
print('El 80% de ejemplos para entrenamiento son la cantidad de: {:.0f} de ejemplos'.format( len(X_train)))
print('El 20% de ejemplos para pruebas son la cantidad de: {:.0f} de ejemplos'.format(  len(X_test)))
print('la catidad total de ejemplos es: {:.0f}'.format( len(dataset)))

    X[:,0]   X[:, 1]   X[:, 2]   X[:, 3]   X[:, 4]   X[:, 5]   X[:, 6]   X[:, 7]   X[:, 8]   X[:, 9]  X[:, 10]  X[:, 11]  X[:, 12]  X[:, 13]  X[:, 14]  X[:, 15]  X[:, 16]  X[:, 17]  X[:, 18]  X[:, 19]         Y
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
         2        12         2         1         0        97         2        20        97         2         2         2         2         2         2         2         2         2         2        97         0
         1        12         2         2         4         2         2        61        97         2         2         2         2         2         2         2         2         2         2         2         1
         2        12         1         1         0        97         2        62         2         2         2      

### 1.4 Normalización de caracteristicas

Al visualizar los datos se puede observar que las caracteristicas tienen diferentes magnitudes, por lo cual se debe transformar cada valor en una escala de valores similares, esto con el fin de que el descenso por el gradiente pueda converger mas rapidamente. Se aplica la normalizacion esto debido a que los datos de las X estan a diferentes escalas.

Hacemos el uso de la siguiente funcion para normalizar los datos de las columnas X:

In [ ]:
def  featureNormalize(X):
    X_norm = X.copy()

    #creamos un array de ceros con una longitud igual al número de columnas en el array X. La variable mu y sigma se inicializa como este array de ceros.
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    #Creamos el promedio de cada columna de X
    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)

    sigma[sigma == 0] = 1

    #normalizamos los datos con la siguiente formula
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

Normalizando los datos:

In [ ]:
X_norm, mu, sigma= featureNormalize(X_train)

print('{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}{:>10s}'.format(
    'X[:,0]', 'X[:, 1]', 'X[:, 2]', 'X[:, 3]', 'X[:, 4]', 'X[:, 5]', 'X[:, 6]', 'X[:, 7]', 'X[:, 8]', 'X[:, 9]', 'X[:, 10]', 'X[:, 11]', 'X[:, 12]',
    'X[:, 13]', 'X[:, 14]', 'X[:, 15]', 'X[:, 16]', 'X[:, 17]', 'X[:, 18]', 'X[:, 19]'
))
print('-' * 250)

for i in range(10):
    print('{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}{:10.3f}'.format(
        X_norm[i, 0],
        X_norm[i, 1],
        X_norm[i, 2],
        X_norm[i, 3],
        X_norm[i, 4],
        X_norm[i, 5],
        X_norm[i, 6],
        X_norm[i, 7],
        X_norm[i, 8],
        X_norm[i, 9],
        X_norm[i, 10],
        X_norm[i, 11],
        X_norm[i, 12],
        X_norm[i, 13],
        X_norm[i, 14],
        X_norm[i, 15],
        X_norm[i, 16],
        X_norm[i, 17],
        X_norm[i, 18],
        X_norm[i, 19],
    ))

    X[:,0]   X[:, 1]   X[:, 2]   X[:, 3]   X[:, 4]   X[:, 5]   X[:, 6]   X[:, 7]   X[:, 8]   X[:, 9]  X[:, 10]  X[:, 11]  X[:, 12]  X[:, 13]  X[:, 14]  X[:, 15]  X[:, 16]  X[:, 17]  X[:, 18]  X[:, 19]
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
     0.763     0.810     1.001    -0.485    -0.281     0.474    -0.113    -1.288     0.994    -0.034    -0.051    -0.047    -0.055    -0.025    -0.065    -0.050    -0.024    -0.050    -0.040     0.474
    -1.311     0.810     1.001     2.060     3.469    -2.104    -0.113     1.136     0.994    -0.034    -0.051    -0.047    -0.055    -0.025    -0.065    -0.050    -0.024    -0.050    -0.040    -2.107
     0.763     0.810    -0.999    -0.485    -0.281     0.474    -0.113     1.195    -1.006    -0.034    -0.051    -0.047    -0.055    -0.216    -0

### 1.5 Creacion de la funcion Sigmoide
También conocida como la función logística, es una función matemática que toma cualquier número real como entrada y devuelve un valor en el rango de 0 a 1. Donde nuestra **Z** es nuestra hipotesis.

Creando la funcion:

In [ ]:
def sigmoid(z):
    z = np.array(z)

    g = np.zeros(z.shape)

    g = 1 / (1 + np.exp(-z))

    return g

### 1.6 Funcion de costo
Aplicando la teoria donde nuestra funcion recibira parametros como Theta, x, y y lamda_, donde lamda_ es nuestro parametro de regularizacion.

Donde la funcion nos devuelve un costo y nuestro gradiente.

In [ ]:
def lrCostFunction(theta, X, y, lambda_):
    #creamos una variable m que contiene la longitud de y
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    #inicializamos J y grad
    J = 0
    grad = np.zeros(theta.shape)

    #calculamos h haciendo uso de la funcion sigmoid, donde h es nuestra hipotesis
    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)

    grad = grad + (lambda_ / m) * temp

    return J, grad

### 1.7 Clasificacion One-vs-All
Aqui estariamos usando un ciclo for para iterar sobre cada una de las clases, luego haciendo uso de `optimize.minimize` que es un método de la biblioteca *scipy* que encuentra el mínimo de una función. En este caso, se trata de minimizar la función de costos de regresión logística `(lrCostFunction)`.

Los parámetros iniciales ``(initial_theta)``.

Una tupla que contiene los datos de entrenamiento ``(X)``, las etiquetas ``(y == c)``, y el parámetro de regularización ``(lambda_)``.

``jac=True`` indica que la función de coste devuelve tanto el coste como el gradiente.

El método de optimización ``('CG' significa Gradiente Conjugado).``

El diccionario de opciones ``(options)``  que se establece en 1000 para limitar el número máximo de iteraciones del optimizador.

In [ ]:
def oneVsAll(X, y, num_labels, lambda_):
    # algunas variables utiles
    #m es la longitud de y, n es la cantidad de columnas en X
    m, n = X.shape

    # inicializamos la matriz de thetas con ceros, y con n+1 columnas que seria nuestro sesgo
    all_theta = np.zeros((num_labels, n + 1))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    # iteramos sobre cada etiqueta (clase) y entrenamos un clasificador
    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 3000}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

Inicializamos nuestra lambda con valor de *0.005*, usamos la funcion de `oneVsAll` donde pasamos los parametros de *X_norm*, *num_labels* que seria la cantidad de clases que tenemos, y nuestro *lanbda* para asi obtener nuestros Thetas para cada clase.

In [ ]:
#defimos el valor de lambda
lambda_ = 100
all_theta = oneVsAll(X_norm, y_train, num_labels, lambda_)
print(all_theta.shape)

(7, 21)


Imprimimos los thetas obtenidos para cada clase:

In [ ]:
for i in range(all_theta.shape[1]):
    print("Thetha",i," para la clase 0:", all_theta[0,i]," ; "
        , "Thetha",i," para la clase 1:", all_theta[1,i]," ; "
        , "Thetha",i," para la clase 2:", all_theta[2,i]," ; "
        , "Thetha",i," para la clase 3:", all_theta[3,i]," ; "
        , "Thetha",i," para la clase 4:", all_theta[4,i]," ; "
        , "Thetha",i," para la clase 5:", all_theta[5,i]," ; "
        , "Thetha",i," para la clase 6:", all_theta[6,i])

Thetha 0  para la clase 0: -0.11882877045539043  ;  Thetha 0  para la clase 1: -4.948437709328698  ;  Thetha 0  para la clase 2: -9.404773388685825  ;  Thetha 0  para la clase 3: -0.9026587751616251  ;  Thetha 0  para la clase 4: -6.137999498143922  ;  Thetha 0  para la clase 5: -3.7146448580906806  ;  Thetha 0  para la clase 6: -2.1568978774776264
Thetha 1  para la clase 0: 0.11423903883190208  ;  Thetha 1  para la clase 1: 0.2220757440957278  ;  Thetha 1  para la clase 2: 0.04237871285235841  ;  Thetha 1  para la clase 3: 0.020036387006895404  ;  Thetha 1  para la clase 4: 0.22865594763562147  ;  Thetha 1  para la clase 5: 0.2460403147005177  ;  Thetha 1  para la clase 6: -0.3558552347905516
Thetha 2  para la clase 0: 0.2376322138028477  ;  Thetha 2  para la clase 1: 0.40056402622454385  ;  Thetha 2  para la clase 2: -0.22718303575722062  ;  Thetha 2  para la clase 3: -0.06158458764429049  ;  Thetha 2  para la clase 4: 0.8414286393065317  ;  Thetha 2  para la clase 5: 0.1020387139297

#### 1.7.1 Prediccion One-vs-all
Aqui creamos solo la funcion, donde mandamos los parametros de `all_theta` y la `X` que en este caso puede ser las X de prueba, pero antes deben de estar normalizadas.

In [ ]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # añadimos unos a la matriz de X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

### 1.8 Validaciones

Para este caso de validacion se hizo uso de la funcion de `predictOneVsAll()` creada anteriormente, los siguiente fue normalizar nuestra `X_test` que son el 20% para pruebas, ahora se procedio a normalizarlo haciendo uso de *mu* y *sigma* calculado anteriormente en la funcion de normalizacion.

luego se hace uso de `np.mean` donde nos calcula el promedio de los valores booleanos en el array resultante de la comparación.

#### 1.8.1 Con los datos de entrenamiento
Para este caso se uso los datos de `X_norm` usando la funcion de `predictOneVsAll()`, viendo el resultado de precision nos da un **53.69%** el cual esta muy lejos de ser un buen modelo.

In [ ]:
# print(X_test.shape)
pred_test = predictOneVsAll(all_theta, X_norm)
print('Precision del conjunto de entrenamiento: {:.2f}%'.format(np.mean(pred_test == y_train) * 100))

Precision del conjunto de entrenamiento: 53.69%


#### 1.8.2 con los datos de prueba
Para este caso se uso los datos de `X_test`, luego se procedio a normalizar los datos.

Usando la funcion de `predictOneVsAll()`, viendo el resultado de precision nos da un **53.62%** , un poco en diferencia a la anterior prediccion.

In [ ]:
X_test1 = X_test.copy()
# print(X_test.shape)
#normalizamos los datos de prueba
X_test1 = (X_test1 - mu) / sigma

pred_test = predictOneVsAll(all_theta, X_test1)
print('Precision del conjunto de entrenamiento: {:.2f}%'.format(np.mean(pred_test == y_test) * 100))

Precision del conjunto de entrenamiento: 53.62%


En este caso se puede hacer las predicciones usando directamente la funcion `sigmoid()`, y luego imprimimos 30 predicciones para ver que tanto se asemejan con nuestra Y predicha.

In [ ]:

#usando el mu y sigma calculado anteriormente, para realizar la normalizacion de los datos de prueba
X_test1 = (X_test1 - mu) / sigma
X_test1 = np.concatenate([np.ones((len(X_test1), 1)), X_test1], axis=1)

print(X_test1.shape)

#para este ejemplo, vamos a mostrar las predicciones de las primeras 30 predicciones
p = np.argmax(sigmoid(X_test1.dot(all_theta.T)), axis = 1)

for i in range(10):
    print(f"Prediccion: {p[i]}, Real: {y_test[i]}")

(209719, 21)
Prediccion: 3, Real: 5
Prediccion: 6, Real: 0
Prediccion: 6, Real: 3
Prediccion: 6, Real: 3
Prediccion: 6, Real: 3
Prediccion: 6, Real: 0
Prediccion: 6, Real: 3
Prediccion: 6, Real: 0
Prediccion: 6, Real: 0
Prediccion: 3, Real: 3
